### __Handwritting Recognition Using TensorFlow and Neural Networks__

#### Author: _Muhammad Taimoor Khan_

##### 1. __Importing Libraries__

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

##### 2. __Test Harness Function__

In [13]:
def run_test():
    trainX, trainY, testX, testY = load_dataset() #loading the dataset
    trainX, testX = prep_pixels(trainX, testX) #prepare pixel data
    scores, histories = evaluate_models(trainX, trainY) #evaluate model
    summarize_diag(histories) #learning curves
    summarize_perf(scores) #summarized estimated performance

##### 3. __Load, Train, and Test dataset from the MNIST Dataset__

In [3]:
def load_dataset():
    (trainX,trainY), (testX,testY) = mnist.load_data() #load the dataset
    #reshape the dataset
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    #one hot code encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testX)
    return trainX, trainY, testX, testY

##### 4. __Scale The Pixels__

In [4]:
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    
    train_norm = train_norm/255.0
    test_norm = test_norm/255.0
    
    return train_norm,test_norm

##### 5. __Define the CNN Model__

In [5]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(28,28,1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    
    #now compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorial_crossentropy', metrics=['accuracy'])
    return model

##### 6. __Evaluate the Model Using K-Fold Cross-Validation__

In [6]:
def evaluate_models(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    #preparing cross-validations
    KFold = KFold(n_folds, shuffle=True, random_state=1)
    #enumrate splits
    for train_ix, test_ix in KFold.split(dataX):
        model = define_model()
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        _, accuracy = model.evaluate(testX, testY, verbose=0)
        
        print('> %.3f ' % (accuracy * 100.0))
        #store scores 
        scores.append(accuracy)
        histories.append(history)
    
    return scores, histories 

##### 7. __Plotting Learning Curves__

In [7]:
def summarize_diag(histories):
    for i in range(len(histories)):
        plt.subplot(2, 1, 1)
        plt.title('Cross Entropy Loss')
        plt.plot(histories[i].history['loses'], color='blue', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', label='test')
        
        plt.subplot(2,1,2)
        plt.title('Classification Accuracy')
        plt.plot(histories[i].history['loses'], color='blue', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', label='test')
        
    plt.show()

##### 8. __Summarize Model Performance__

In [8]:
def summarize_perf(scores):
    print('Accuracy = mean=%.3f std=%.3f n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
    
    plt.boxplot(scores)
    plt.show()

##### 9. __Running Test Harness__

In [14]:
run_test()

MemoryError: Unable to allocate 7.48 GiB for an array with shape (7840000, 256) and data type float32

#### <code> The Approx Accuracy is 97% </code>